In [6]:
%matplotlib notebook
import firedrake
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np
from hydropack.constants import *
from hydropack.models.glads import *

SyntaxError: invalid syntax (phi_solver.py, line 114)

In [ ]:
in_dir = "inputs_slope/"
# Output directory
out_dir = "out/"
# Process number
MPI_rank = MPI.rank(mpi_comm_world())

# Load mesh and create function spaces
mesh = Mesh(in_dir + "mesh.xml")
V_cg = FunctionSpace(mesh, "CG", 1)
V_cr = FunctionSpace(mesh, "CR", 1)

In [ ]:
# Initial sheet height
h_init = Function(V_cg)
h_init.interpolate(Constant(0.05))

In [ ]:
# Initial channel areas
S_init = Function(V_cr)

# Initial potential
phi_init = Function(V_cg)
File(in_dir + "phi_0.xml") >> phi_init

# Load the boundary facet function
boundaries = FacetFunction('size_t', mesh)
File(in_dir + "boundaries.xml") >> boundaries

# Load potential at 0 pressure
phi_m = Function(V_cg)
File(in_dir + "phi_m.xml") >> phi_m

# Enforce 0 pressure bc at margin
bc = DirichletBC(V_cg, phi_m, boundaries, 1)


In [ ]:
model_inputs = {}
model_inputs['mesh'] = mesh
model_inputs['h_init'] = h_init
model_inputs['S_init'] = S_init
model_inputs['phi_init'] = phi_init
model_inputs['d_bcs'] = [bc]
model_inputs['maps_dir'] = in_dir + "maps/"
model_inputs['out_dir'] = out_dir

# Create the Glads model
model = Glads(model_inputs, in_dir)

In [ ]:
# Seconds per day
spd = pcs['spd']
# End time
T = 10.0 * spd
# Time step
dt = 60.0 * 30.0
# Irataion count
i = 0

while model.t < T:
  if MPI_rank == 0: 
    current_time = model.t / spd
    print "Current Time: " + str(current_time)
  
  model.step(dt)
  
  if i % 24 == 0:
    model.write_pvds()
  
  if MPI_rank == 0: 
    print